In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date

from bs4 import BeautifulSoup
import requests
import csv
from random import randint
from time import sleep

In [2]:
with open('links.csv', newline='') as f:
    reader = csv.reader(f)
    your_list = list(reader)
links = []
for i in range(len(your_list)):
    links.append(your_list[i][0])
links = list(set(links))

In [3]:
today = date.today()

def date_parse(script, day1, day2):
    parse = script[script.index(day1) + len(day1) + 2 : script.index(day2)-2]
    return parse.replace('[','').replace(']','').replace('false', "0").replace('true', "1").replace(',','')

def date_add(date_str):
        count = 0
        for i in date_str:
            count += int(i)
        return count
    
def make_series(value, series_name):
    return pd.Series(value, name=series_name)

In [4]:
def scrap_url(url):
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html')
    soup_text = soup.prettify()
    
    try:
        description = soup.find('p', {"class":"spc-tiny medium-hide secondary-text light-header"}).text
    except:
        description = ''
    
    try:
        hour_str = soup.find('h3').text
        hours_tutor = int(hour_str[:hour_str.index(' hours')].replace(',',""))
    except: 
        hours_tutor = ''
    try:
        rate_str = soup.find('h4').div.contents[0]
        rate = int(rate_str[rate_str.index('$')+1:])
    except: 
        rate = ''
    
    try:   
        bio = ''
        for tag in soup.find_all('div', {"class":"full-content hide"}):
            bio = bio + tag.text
            bio = bio.replace('\n',"")
    except:
        bio = ''
    
    try:
        edu = []
        for tag in soup.find_all('section', {"class":"spc-med-s"}):
            item = str(tag.text).replace('\n', '').strip()
            item = " ".join(item.split())
            edu.append(item)
    except:
        edu = ''
        
    try:
        subjects_emph = set()
        for i in soup.find_all('a', {"class":"text-nowrap"}):
            item = str(i.text).replace('\n', '').strip()
            item = item.replace(',', '')
            item = " ".join(item.split())
            subjects_emph.add(item)
        subjects_emph.discard('Google Play')
        subjects_emph.discard('App Store')
        
    except:
        subjects_emph = ''
        
    try:
        subjects_all = set()
        for i in soup.find_all('span', {'class':'text-nowrap'}):
            item = str(i.text).replace('\n', '').strip()
            item = item.replace(',', '')
            item = " ".join(item.split())
            subjects_all.add(item)
    except:
        subjects_all = ''
        
    try:
        ratings_text = soup.find('a',{"class":"ratings-anchor"}).text.replace('(','').replace(')','').replace(',', '')
        ratings_split = ratings_text.split()
        num_ratings = int(ratings_split[0])
        num_unique_students = hours_tutor / num_ratings
    except:
        ratings_text = ''
        num_ratings = ''
        num_unique_students = ''
           
    try:
        cal = soup.find('div',{"class":"availabilities-container hide"}).script.text
        sun = date_parse(cal, 'sunday', 'monday')
        mon = date_parse(cal, 'monday', 'tuesday')
        tues = date_parse(cal, 'tuesday', 'wednesday')
        wed = date_parse(cal, 'wednesday', 'thursday')
        thurs = date_parse(cal, 'thursday', 'friday')
        fri = date_parse(cal, 'friday', 'saturday')
        sat = cal[cal.index('saturday') + 10 : -2].replace('[','').replace(']','').replace('false', "0").replace('true', "1").replace(',','')
        hours_sun = date_add(sun)
        hours_mon = date_add(mon)
        hours_tues = date_add(wed)
        hours_wed = date_add(wed)
        hours_thurs = date_add(thurs)
        hours_fri = date_add(fri)
        hours_sat = date_add(sat)
        hours_per_week_schedule = hours_sun + hours_mon + hours_tues + hours_wed + hours_thurs + hours_fri + hours_sat
        
    except:
        hours_sun = ''
        hours_mon = ''
        hours_tues = ''
        hours_wed = ''
        hours_thurs = ''
        hours_fri = ''
        hours_sat = ''
        hours_per_week_schedule = ''
        
    try:    
        bg_check = soup.find_all('p',{"class":'spc-zero'})[1].text
        bg_check_date = bg_check.replace('\n', '').split()[4]
        bg_check_date = datetime.strptime(bg_check_date, '%m/%d/%Y').date()
    
    except:
        bg_check_date = date.today()
    
    try:
        review_dates = []
        for i in range(len(soup_text)):
            if soup_text[i:i+11] == 'dateCreated':
                review_dates.append(soup_text[i:i+24])
    except:
        review_dates = ''
        
    descriptions = pd.Series(description, name='descriptions')
    hours_tutor = pd.Series(hours_tutor, name='hours_tutor')
    rate = pd.Series(rate, name='rate')
    bio = pd.Series(bio, name='bio')
    edu = pd.Series([edu], name='edu')
    subjects_emph = pd.Series([list(subjects_emph)], name='subjects_emph')
    subjects_all = pd.Series([list(subjects_all)], name='subjects_all')
    num_ratings = pd.Series(num_ratings, name='num_ratings')
    hours_sun = pd.Series(hours_sun, name='hours_sun')
    hours_mon = pd.Series(hours_mon, name='hours_mon')
    hours_tues = pd.Series(hours_tues, name='hours_tues')
    hours_wed = pd.Series(hours_wed, name='hours_wed')
    hours_thurs = pd.Series(hours_thurs, name='hours_thurs')
    hours_fri = pd.Series(hours_fri, name='hours_fri')
    hours_sat = pd.Series(hours_sat, name='hours_sat')
    hours_per_week_schedule = pd.Series(hours_per_week_schedule, name='hours_per_week_schedule')
    bg_check_date = pd.Series(bg_check_date, name='bg_check_date')
    review_dates = pd.Series([list(review_dates)], name='review_dates')
    
    scraped_data = pd.concat([descriptions, hours_tutor, rate, bio, num_ratings, subjects_emph, subjects_all, edu,
                       hours_sun, hours_mon, hours_tues, hours_wed, hours_thurs, hours_fri, hours_sat,
                       hours_per_week_schedule, bg_check_date, review_dates], axis=1)     
    return scraped_data

In [ ]:
%%time
df = pd.DataFrame()
#for i in range(200):
for i in range(len(links)):
    scraped = scrap_url(links[i])
    df = df.append(scraped)
    sleep(randint(5,10))

df.reset_index(drop=True, inplace=True)
length_dates = [len(i) for i in df['review_dates']]
for i in range(len(length_dates)):
    if length_dates[i] == 0:
        df.drop(i, inplace=True)
#df.drop_duplicates(inplace=True)
#df.to_csv('test_tutor_data.csv')
df.to_csv('tutor_data.csv')

In [ ]:
#drop missing column values
for i in ['descriptions', 'num_ratings', 'hours_tutor', 'rate', 'hours_per_week_schedule']:
    df.drop(df[df[i] == ''].index, inplace=True)

In [150]:
# generate target hours_per_week_estimate
today = date.today()
earliest_review_date = []
for i in df['review_dates']:
    list_dates = []
    for d in i:
        list_dates.append(datetime.strptime(d[14:], '%Y-%m-%d').date())
    earliest_review = min(list_dates)
    earliest_review_date.append(earliest_review)
    #latest_review = max(c)
df['earliest_review_date'] = np.array(earliest_review_date)
df['earliest_date'] = np.minimum(df['bg_check_date'], df['earliest_review_date'])
days_since_start = (today - df['earliest_date']).dt.days
df['hours_per_week_estimate'] = (df['hours_tutor'] / days_since_start) * 7 

In [151]:
#additional length features
length_checker = np.vectorize(len)
df['descriptions_length'] = length_checker(df['descriptions']) 
df['bio_length'] = length_checker(df['bio']) 
df['number_subjects'] = length_checker(df['subjects_emph']) +  length_checker(df['subjects_all'])
df['number_degrees'] = length_checker(df['edu'])

In [154]:
# columns to drop
cols_to_drop = ['hours_tutor', 'num_ratings', 'review_dates', 'earliest_date']
df.drop(columns=cols_to_drop, inplace=True)